In [3]:
import numpy as np 
import pandas as pd
import math
import matplotlib.pyplot as plt 
import scipy.optimize
from scipy.stats import poisson, chi2

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# $$H_0: \xi \ \text{\textasciitilde} \ pois(\theta)$$
# $$H_1: \overline{H_0}$$
# $$\alpha = 0.05$$

## Проверка гипотезы с помощью критерия ${\chi}^2$ (Пирсона) для сложной гипотезы

### Функция правдоподобия для ОМПГ:

## $$L(\overrightarrow{x_n}, \overrightarrow{\theta}) = \displaystyle\prod_{i = 1}^{n} p_i^{m_i},\ \ L(\overrightarrow{x_n}, \overrightarrow{\theta}) \rightarrow \max$$
## $$\Leftrightarrow$$
## $$\ln (L(\overrightarrow{x_n}, \overrightarrow{\theta})) = \displaystyle\sum_{i = 1}^{n} m_i\ln{p_i},\ \ \ln (L(\overrightarrow{x_n}, \overrightarrow{\theta})) \rightarrow \max$$

#### $L(\vec{x_n},\ \theta) = \bigg(e^{-\theta}\bigg)^{109} \bigg(\theta e^{-\theta}\bigg)^{65} \bigg(\frac{\theta^2 e^{-\theta}}{2}\bigg)^{22} \bigg(\frac{\theta^3 e^{-\theta}}{6}\bigg)^3 \bigg(\frac{\theta^4 e^{-\theta}}{24}\bigg)^1 \bigg(1 - \displaystyle\sum_{k=0}^4 p_k\bigg)^0$
#### $lnL = 122ln\theta - 200\theta + C$
#### $(lnL)' = \frac{122}{\theta} - 200$ $\Rightarrow$ $\widetilde{\theta} = \frac{61}{100}$ 
#### $(lnL)'' = -\frac{122}{\theta^2} > 0$ $\Rightarrow$ $\widetilde{\theta}$ - точка максимума $L(\theta)$ 

In [4]:
theta = 0.61
n = 200
m_i = np.array([109, 65, 22, 3, 1, 0])
p_i = np.zeros(6)
np_i = np.zeros(6)

for i in range(0, 5):
    prob = poisson.pmf(i, theta)
    p_i[i] = prob
    _np = prob*n
    np_i[i] = _np
prob = 1 - np.sum(p_i[0:4])
p_i[5] = prob
_np = prob*n
np_i[5] = _np

df = pd.DataFrame(
    columns=["[0, 1)", "[1, 2)", "[2, 3)", "[3, 4)", "[4, 5)", "[5, +inf)"],
    index=pd.Index(["m_i", "p_i", "np_i"]))
df.loc["m_i", :] = m_i
df.loc["p_i", :] = p_i
df.loc["np_i", :] = np_i
df

,"[0, 1)","[1, 2)","[2, 3)","[3, 4)","[4, 5)","[5, +inf)"
m_i,109,65,22,3,1,0
p_i,0.543351,0.331444,0.10109,0.020555,0.003135,0.00356
np_i,108.670174,66.288806,20.218086,4.111011,0.626929,0.711924


#### $np_4 < 5,\ np_5 < 5,\ \Rightarrow$ объединяем [2, 3), [3, 4) [4, 5) и [5, $+\infty$) в [2, $+\infty$)

#### $L(\theta) = \bigg(e^{-\theta}\bigg)^{109} \bigg(\theta e^{-\theta}\bigg)^{65}\bigg(1 - e^{-\theta} - \theta e^{-\theta}\bigg)^{26}$
#### $lnL(\theta) = 65ln\theta - 174\theta + 26ln(1 - e^{\theta} - \theta e^{-\theta})$

### Считаем $\widetilde{\theta}$ с помощью scipy.optimize.minimize_scalar

In [5]:
const = 1
def lnL(thetta: float) -> float:
    return 109*math.log(thetta) - 196*thetta + \
    4*math.log(1 - math.exp(-thetta) - thetta*math.exp(-thetta) - ((thetta**2)/2)*math.exp(-thetta)) + const
def lnL1(thetta: float) -> float:
    return 122*math.log(thetta) - 200*thetta + const
def lnL2(thetta: float) -> float:
    return 113*math.log(1 - thetta*math.exp(-thetta) - ((thetta**2)/2)*math.exp(-thetta)) + \
    109*math.log(thetta) - 87*thetta
def lnL3(thetta: float) -> float:
    return 65*math.log(thetta) - 174*thetta + 26*math.log(1 - math.exp(-thetta) - thetta*math.exp(-thetta))
thetta_3 = scipy.optimize.minimize_scalar(lambda x: (-1)*lnL3(x), bounds=(0, 100), method="bounded").x
print(f"thetta_3 = {thetta_3}")

thetta_3 = 0.6144599577791148


In [6]:
theta = 0.614
n = 200
m_i = np.array([109, 65, 26])
p_i = np.zeros(3)
np_i = np.zeros(3)
delta = 0 

for i in range(0, 2):
    prob = poisson.pmf(i, theta)
    p_i[i] = prob
    _np = prob*n
    np_i[i] = _np
    delta += ((m_i[i] - _np)**2)/_np

prob = 1 - np.sum(p_i[0:2])
p_i[2] = prob
_np = prob*n
np_i[2] = _np
delta += ((m_i[2] - _np)**2)/_np

df = pd.DataFrame(
    columns=["[0, 1)", "[1, 2)", "[2, +inf)"],
    index=pd.Index(["m_i", "p_i", "np_i"]))
df.loc["m_i", :] = m_i
df.loc["p_i", :] = p_i
df.loc["np_i", :] = np_i
df

,"[0, 1)","[1, 2)","[2, +inf)"
m_i,109,65,26
p_i,0.541182,0.332286,0.126533
np_i,108.236361,66.457126,25.306513


## $$\widetilde{\Delta} = \displaystyle\sum_{i=1}^n \frac{(n p_i - m_i)^2}{n p_i}$$

In [ ]:
print(f"\u0303\u0394 = {delta}")

delta = 0.056340325026625865


In [8]:
sf = chi2.sf(delta, 1)
print(sf)

0.8123766108594808
